## Настройки Colab

In [1]:
# Настройка пользователя (сделать один раз)
!git config --global user.email "nabludatellip@gmail.com"
!git config --global user.name "ProninPV"

In [2]:
!git clone https://github.com/ProninPV/ml-regression_concrete-strength.git
%cd ml-regression_concrete-strength


Cloning into 'ml-regression_concrete-strength'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 107 (delta 21), reused 104 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 1.26 MiB | 7.52 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/ml-regression_concrete-strength


In [ ]:
%cd /content/ml-regression_concrete-strength
!git config pull.rebase false
!git pull origin modeling

In [ ]:
# Создай коммит слияния
!git add .
!git commit -m "Merge remote changes"

# Затем пуш
!git push origin modeling

In [ ]:
%cd /content/ml-regression_concrete-strength

# Добавь ноутбук в git
!git add notebooks/05_Modeling_colab.ipynb

# Закоммить
!git commit -m "Update modeling notebook"

# Затем пушить


In [ ]:
%cd /content/ml-regression_concrete-strength

# Если есть изменения - добавить и закоммитить
!git add .
!git commit -m "Your commit message"

# Затем пушить


In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.9 MB/s eta 0:00:00


## 6.0 Импорты библиотек

In [1]:
import os
import yaml
import logging
import pickle
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene
from scipy.stats import ttest_ind
from typing import List, Any, Optional, Tuple, Dict, Union
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import time
import psutil
from tqdm import tqdm
import gc

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [11]:
# В Colab проект клонируется в /content/
# Устанавливаем правильную рабочую директорию
# project_root = Path('/content/ml-regression_concrete-strength')

# Устанавливаем правильную рабочую директорию для работы локально
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Меняем рабочую директорию на корень проекта
os.chdir(project_root)

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Проверяем наличие конфиг файла
config_path = project_root / "config" / "config.yaml"
print(f"Looking for config at: {config_path}")

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving
from src.features import feat_preprocessing
from src.modeling import modeling

config = loader.load_config(config_path)
print("✅ Config loaded successfully!")

Looking for config at: D:\Skills\Kaggle\ml-regression_concrete-strength\config\config.yaml
✅ Config loaded successfully!


## 6.1. Загрузка данных

In [12]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train',
                                         config=config)

[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [13]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


In [14]:
# Загрузка test
df_test = loader.data_load_preprocessed(data_type='test', config=config)

[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_test.pkl
[✓] Данные успешно загружены. Форма: (228, 10)


In [15]:
# Вывод первых 5 строк тестового датасета
df_test.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,W/C,Sp/C_pct
0,167.4,129.9,128.6,175.5,7.8,1006.3,746.6,28,1.048387,0.046595
1,475.0,118.8,0.0,181.1,8.9,852.1,781.5,7,0.381263,0.018737
2,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100,0.749801,0.025457
3,307.0,0.0,0.0,193.0,0.0,968.0,812.0,365,0.628664,0.000000
4,143.6,0.0,174.9,158.4,17.9,942.7,844.5,28,1.103064,0.124652


## 6.2. Предобработка данных

In [14]:
# Разделение на признаки и целевую переменную
X = df_train.drop('Strength', axis=1)
y = df_train['Strength']
y_name = y.name

## 6.3. Подбор гипепараметров для Catboost

#### Объявление функций

#### Тюнинг Catboost

## 6.4. Подбор гипепараметров для 

## Отправка на Github

In [32]:
from getpass import getpass

# 1. Безопасный ввод токена
GITHUB_TOKEN = getpass('Введите ваш GitHub Personal Access Token: ')

Введите ваш GitHub Personal Access Token: ··········


In [33]:
!git status

On branch modeling
Your branch is up to date with 'origin/modeling'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   catboost_info/catboost_training.json
	modified:   catboost_info/learn/events.out.tfevents
	modified:   catboost_info/time_left.tsv
	modified:   data/processed/data_train_outliers.csv
	modified:   data/processed/data_train_outliers.parquet
	modified:   data/processed/data_train_outliers.pkl
	modified:   data/processed/y_train_outliers.pkl

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	models/modeling_report/modeling_experiments_20251115_121552.csv
	models/pipelines/best_pipeline_CatBoostRegressor.pkl
	models/pipelines/pipeline_metadata_CatBoostRegressor.pkl

no changes added to commit (use "git add" and/or "git commit -a")


In [34]:
# 1. Добавляем файлы
!git add .

# 2. Коммитим
!git commit -m "feat: Colab experiment with data preprocessing strategies - Train multiple models using different data preprocessing pipelines - Save the best performing pipeline and experiment report"

[modeling a511430] feat: Colab experiment with data preprocessing strategies - Train multiple models using different data preprocessing pipelines - Save the best performing pipeline and experiment report
 10 files changed, 395 insertions(+), 205 deletions(-)
 rewrite catboost_info/catboost_training.json (97%)
 rewrite catboost_info/learn/events.out.tfevents (95%)
 rewrite catboost_info/time_left.tsv (97%)
 rewrite data/processed/data_train_outliers.parquet (78%)
 rewrite data/processed/data_train_outliers.pkl (90%)
 rewrite data/processed/y_train_outliers.pkl (76%)
 create mode 100644 models/modeling_report/modeling_experiments_20251115_121552.csv
 create mode 100644 models/pipelines/best_pipeline_CatBoostRegressor.pkl
 create mode 100644 models/pipelines/pipeline_metadata_CatBoostRegressor.pkl


In [35]:
!git push https://{GITHUB_TOKEN}@github.com/ProninPV/ml-regression_concrete-strength.git modeling

Enumerating objects: 33, done.
Counting objects: 100% (33/33), done.
Delta compression using up to 2 threads
Compressing objects: 100% (18/18), done.
Writing objects: 100% (19/19), 34.95 KiB | 3.50 MiB/s, done.
Total 19 (delta 7), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (7/7), completed with 6 local objects.
To https://github.com/ProninPV/ml-regression_concrete-strength.git
   51369d6..a511430  modeling -> modeling
